In [1]:
#!/usr/bin/env python
from __future__ import print_function


import os
##disable graphics for cloud computing
os.putenv('SDL_VIDEODRIVER', 'fbcon')
os.environ["SDL_VIDEODRIVER"] = "dummy"
import argparse
import skimage as skimage
from skimage import transform, color, exposure
from skimage.transform import rotate
from skimage.viewer import ImageViewer
import sys
sys.path.append("game/")
import wrapped_flappy_bird as game
import random
import numpy as np
from collections import deque

/anaconda2/lib/python2.7/site-packages/skimage/viewer/utils/core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


In [2]:

import json
from keras import initializers
from keras.initializers import normal, identity
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD , Adam
import tensorflow as tf

Using TensorFlow backend.


In [3]:
GAME = 'bird' # the name of the game being played for log files
CONFIG = 'nothreshold'
ACTIONS = 2 # number of valid actions
GAMMA = 0.99 # decay rate of past observations
OBSERVATION = 3200. # timesteps to observe before training
EXPLORE = 1000#3000000. # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 100#50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4

In [4]:
img_rows , img_cols = 80, 80
#Convert image into Black and white
img_channels = 4 #We stack 4 frames

def buildmodel():
    print("Now we build the model")
    model = Sequential()
    model.add(Convolution2D(32, 8, 8, subsample=(4, 4), border_mode='same',input_shape=(img_rows,img_cols,img_channels)))  #80*80*4
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 4, 4, subsample=(2, 2), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode='same'))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(2))
   
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    print("We finish building the model")
    return model

In [27]:
def trainNetwork(model,args='train'):
    # open up a game state to communicate with emulator
    game_state = game.GameState()

    # store the previous observations in replay memory
    D = deque()

    # get the first state by doing nothing and preprocess the image to 80x80x4
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] = 1
    x_t, r_0, terminal = game_state.frame_step(do_nothing)

    x_t = skimage.color.rgb2gray(x_t)
    x_t = skimage.transform.resize(x_t,(80,80))
    x_t = skimage.exposure.rescale_intensity(x_t,out_range=(0,255))

    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
    #print (s_t.shape)

    #In Keras, need to reshape
    s_t = s_t.reshape(1, s_t.shape[0], s_t.shape[1], s_t.shape[2])  #1*80*80*4

    

    if args == 'Run':
        print('running with saved model')
        OBSERVE = 999999999    #We keep observe, never train
        epsilon = FINAL_EPSILON
        print ("Now we load weight")
        model.load_weights("model.h5")
        adam = Adam(lr=LEARNING_RATE)
        model.compile(loss='mse',optimizer=adam)
        print ("Weight load successfully")    
    else:                       #We go to training mode
        OBSERVE = OBSERVATION
        epsilon = INITIAL_EPSILON

    t = 0
    while (True):
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0
        a_t = np.zeros([ACTIONS])
        #choose an action epsilon greedy
        if t % FRAME_PER_ACTION == 0:
            if random.random() <= epsilon:
                print("----------Random Action----------")
                action_index = random.randrange(ACTIONS)
                a_t[action_index] = 1
            else:
                q = model.predict(s_t)       #input a stack of 4 images, get the prediction
                max_Q = np.argmax(q)
                action_index = max_Q
                a_t[max_Q] = 1

        #We reduced the epsilon gradually
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        #run the selected action and observed next state and reward
        x_t1_colored, r_t, terminal = game_state.frame_step(a_t)

        x_t1 = skimage.color.rgb2gray(x_t1_colored)
        x_t1 = skimage.transform.resize(x_t1,(80,80))
        x_t1 = skimage.exposure.rescale_intensity(x_t1, out_range=(0, 255))

        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1) #1x80x80x1
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3)

        # store the transition in D
        D.append((s_t, action_index, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        #only train if done observing
        if t > OBSERVE:
            #sample a minibatch to train on
            minibatch = random.sample(D, BATCH)



            inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))   #32, 80, 80, 4
            print (inputs.shape)
            targets = np.zeros((inputs.shape[0], ACTIONS))                         #32, 2

            #Now we do the experience replay
            for i in range(0, len(minibatch)):
                state_t = minibatch[i][0]
                action_t = minibatch[i][1]   #This is action index
                reward_t = minibatch[i][2]
                state_t1 = minibatch[i][3]
                terminal = minibatch[i][4]
                # if terminated, only equals reward

                inputs[i:i + 1] = state_t    #I saved down s_t

                targets[i] = model.predict(state_t)  # Hitting each buttom probability
                Q_sa = model.predict(state_t1)

                if terminal:
                    targets[i, action_t] = reward_t
                else:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)

            # targets2 = normalize(targets)
            loss += model.train_on_batch(inputs, targets)

        s_t = s_t1
        t = t + 1

        # save progress every 10000 iterations
        if t % 1000 == 0:
            print("Now we save model")
            model.save_weights("model.h5", overwrite=True)
            with open("model.json", "w") as outfile:
                json.dump(model.to_json(), outfile)

        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state, \
            "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t, \
            "/ Q_MAX " , np.max(Q_sa), "/ Loss ", loss)

    print("Episode finished!")
    print("************************")

In [25]:
def playGame(args='train'):
    model = buildmodel()
    trainNetwork(model,args)

def main():
    parser = argparse.ArgumentParser(description='Description of your program')
    parser.add_argument('-m','--mode', help='Train / Run', required=True)
    args = vars(parser.parse_args())
    playGame(args)

In [24]:
playGame()

Now we build the model
We finish building the model


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), padding="same", strides=(4, 4), input_shape=(80, 80, 4...)`
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), padding="same", strides=(2, 2))`
  # Remove the CWD from sys.path while we load stuff.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", strides=(1, 1))`
  if sys.path[0] == '':


TIMESTEP 1 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 2 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
----------Random Action----------
TIMESTEP 3 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 4 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 5 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 6 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
----------Random Action----------
TIMESTEP 7 / STATE observe / EPSILON 0.1 / ACTION 1 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 8 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 9 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 10 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 11 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_M

TIMESTEP 94 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 95 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 96 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 97 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 98 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 99 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD -1 / Q_MAX  0 / Loss  0
TIMESTEP 100 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 101 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 102 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 103 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 104 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
----------Random Action----------
TIMES

TIMESTEP 186 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 187 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 188 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 189 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 190 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 191 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 192 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 193 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 194 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
----------Random Action----------
TIMESTEP 195 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 196 / STATE observe / EPSILON 0.1 / ACTION 0 / REWARD -1 / Q_MAX  0 / Loss  0

...

TIMESTEP 5025 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5026 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5027 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5028 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5029 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5030 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5031 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5032 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5033 / STATE train / EP

TIMESTEP 5093 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5094 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD -1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5095 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5096 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5097 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5098 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5099 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5100 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5101 / STATE train / EPS

TIMESTEP 5161 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5162 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5163 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5164 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5165 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5166 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5167 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5168 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5169 / STATE train / EP

TIMESTEP 5229 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5230 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5231 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5232 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5233 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5234 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5235 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5236 / STATE train / EPSILON 1.00000000017e-07 / ACTION 0 / REWARD 0.1 / Q_MAX  nan / Loss  nan
(32, 80, 80, 4)
TIMESTEP 5237 / STATE train / EP

KeyboardInterrupt: 

In [28]:
playGame(args='Run')

Now we build the model
We finish building the model

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (8, 8), padding="same", strides=(4, 4), input_shape=(80, 80, 4...)`
  
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (4, 4), padding="same", strides=(2, 2))`
  # Remove the CWD from sys.path while we load stuff.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", strides=(1, 1))`
  if sys.path[0] == '':



running with saved model
Now we load weight
Weight load successfully
TIMESTEP 1 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 2 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 3 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 4 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 5 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 6 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 7 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 8 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 9 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 10 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 11 / STATE observe / EPSILON 0

TIMESTEP 95 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 96 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 97 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 98 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 99 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 100 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 101 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 102 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 103 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 104 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 105 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIME

TIMESTEP 187 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 188 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 189 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD -1 / Q_MAX  0 / Loss  0
TIMESTEP 190 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 191 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 192 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 193 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 194 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 195 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 196 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 197 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0


...

TIMESTEP 1477 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1478 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1479 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1480 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1481 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD -1 / Q_MAX  0 / Loss  0
TIMESTEP 1482 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1483 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1484 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1485 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1486 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1487 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0

TIMESTEP 1568 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1569 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1570 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1571 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1572 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1573 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1574 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1575 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1576 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD -1 / Q_MAX  0 / Loss  0
TIMESTEP 1577 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1578 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0

TIMESTEP 1659 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1660 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1661 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1662 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1663 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1664 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1665 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1666 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1667 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1668 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1669 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  

TIMESTEP 1751 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1752 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1753 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1754 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1755 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1756 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1757 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1758 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1759 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1760 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  0 / Loss  0
TIMESTEP 1761 / STATE observe / EPSILON 0.0001 / ACTION 0 / REWARD 0.1 / Q_MAX  

KeyboardInterrupt: 

In [7]:


if __name__ == "__main__":
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    from keras import backend as K
    K.set_session(sess)
    main()


usage: ipykernel_launcher.py [-h] -m MODE
ipykernel_launcher.py: error: argument -m/--mode is required


SystemExit: 2

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
